In [31]:
import pandas as pd
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv
import time
import json
from typing import Dict, List
import re
from PIL import Image

# Load environment variables
load_dotenv()

# Configure Gemini API
client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

In [32]:
# Định nghĩa các prompt template cho 3 tác vụ phân loại

BLOOM_TAXONOMY_PROMPT = """
Bạn là một chuyên gia giáo dục. Hãy phân loại câu hỏi sau theo thang đo Bloom's Taxonomy:

Câu hỏi: "{question}"
Câu trả lời: "{answer}"

Các mức độ trong Bloom's Taxonomy:
1. Remember (Nhớ): Nhớ lại thông tin đã học
2. Understand (Hiểu): Giải thích ý nghĩa, diễn đạt lại
3. Apply (Áp dụng): Sử dụng kiến thức trong tình huống mới
4. Analyze (Phân tích): Phân tích thành phần, tìm mối quan hệ
5. Evaluate (Đánh giá): Đưa ra đánh giá, phê phán
6. Create (Sáng tạo): Tạo ra điều mới, kết hợp các yếu tố

Hãy trả lời chỉ MỘT từ trong số: Remember, Understand, Apply, Analyze, Evaluate, Create
"""

GRADE_LEVEL_PROMPT = """
Bạn là một chuyên gia giáo dục tiểu học Việt Nam. Hãy phân loại câu hỏi sau phù hợp với lớp nào trong bậc tiểu học (1-5):

Câu hỏi: "{question}"
Câu trả lời: "{answer}"

Tiêu chí phân loại:
- Lớp 1: Câu hỏi đơn giản, nhận biết cơ bản (màu sắc, số lượng đơn giản, tên gọi)
- Lớp 2: Câu hỏi mô tả đơn giản, so sánh cơ bản
- Lớp 3: Câu hỏi yêu cầu tư duy logic đơn giản, phân loại
- Lớp 4: Câu hỏi phân tích, so sánh phức tạp hơn
- Lớp 5: Câu hỏi đánh giá, kết luận, tư duy phản biện

Hãy trả lời chỉ MỘT số: 1, 2, 3, 4, hoặc 5
"""

QUESTION_TYPE_PROMPT = """
Bạn là một chuyên gia phân loại câu hỏi giáo dục. Hãy phân loại câu hỏi sau theo các dạng:

Câu hỏi: "{question}"
Câu trả lời: "{answer}"

Các dạng câu hỏi:
1. LOC (Xác định vị trí): Câu hỏi yêu cầu xác định vị trí, hướng, nơi chốn của một đối tượng trong ảnh
   Ví dụ: "Con mèo đang ở đâu trong bức tranh?" "Người lính đứng ở phía nào của ảnh?"

2. CNT (Đếm số lượng): Câu hỏi yêu cầu đếm số lượng đối tượng giống nhau trong ảnh
   Ví dụ: "Có bao nhiêu con vịt trong hồ?" "Trong lớp học có bao nhiêu học sinh đang đọc sách?"

3. OBJ (Nhận dạng đối tượng): Câu hỏi yêu cầu nhận biết hoặc mô tả một đối tượng cụ thể như tên, đặc điểm, màu sắc, hình dạng
   Ví dụ: "Quả táo có màu gì?" "Người đàn ông đang cưỡi con vật gì?" "Chiếc xe kia là xe gì?"

4. ACT (Miêu tả hành động): Câu hỏi yêu cầu mô tả hành động, hoạt động của người/vật đang diễn ra trong ảnh
   Ví dụ: "Bạn nhỏ đang làm gì?" "Người phụ nữ đang nấu ăn phải không?"

5. INF (Suy luận ngắn): Câu hỏi yêu cầu suy luận dựa trên chi tiết trong ảnh để đưa ra kết luận ngắn gọn
   Ví dụ: "Vì sao trời nhiều mây?" "Bức tranh có thể được vẽ vào mùa nào?"

6. REL (Quan hệ/So sánh): Câu hỏi yêu cầu xác định mối quan hệ, so sánh giữa các đối tượng (to-nhỏ, xa-gần, cao-thấp, trước-sau)
   Ví dụ: "Con vật nào to hơn?" "Cái cây bên phải cao hơn hay thấp hơn?"

7. TXT (Đọc hiểu văn bản): Câu hỏi yêu cầu học sinh phải đọc nội dung chữ trong ảnh để trả lời
   Ví dụ: "Theo bảng, lớp 3A có bao nhiêu học sinh giỏi?" "Tấm biển ghi gì?"

8. OTH (Khác): Câu hỏi không thuộc các loại trên hoặc không đủ thông tin rõ ràng để phân loại
   Ví dụ: "Em thích điều gì trong bức tranh này?" "Bức ảnh này mang lại cảm giác gì?"

Hãy trả lời chỉ MỘT mã trong số: LOC, CNT, OBJ, ACT, INF, REL, TXT, OTH
"""

print("Đã định nghĩa các prompt template!")

Đã định nghĩa các prompt template!


In [33]:
def classify_question(image_path: str, question: str, answer: str, classification_type: str) -> str:
    """
    Phân loại câu hỏi theo loại được chỉ định với đầu vào là ảnh, câu hỏi và câu trả lời
    
    Args:
        image_path: Đường dẫn đến ảnh
        question: Câu hỏi cần phân loại
        answer: Câu trả lời tương ứng
        classification_type: Loại phân loại ('bloom', 'level', 'type')
    
    Returns:
        Kết quả phân loại
    """
    
    # Chọn prompt phù hợp
    if classification_type == 'bloom':
        prompt = BLOOM_TAXONOMY_PROMPT.format(question=question, answer=answer)
    elif classification_type == 'level':
        prompt = GRADE_LEVEL_PROMPT.format(question=question, answer=answer)
    elif classification_type == 'type':
        prompt = QUESTION_TYPE_PROMPT.format(question=question, answer=answer)
    else:
        raise ValueError(f"Loại phân loại không hợp lệ: {classification_type}")
    
    try:
        # Kiểm tra file ảnh có tồn tại không
        if not os.path.exists(image_path):
            print(f"Ảnh không tồn tại: {image_path}")
            return "Unknown"
        
        # Đọc ảnh bằng PIL
        image = Image.open(image_path).convert("RGB")
        
        # Tạo nội dung cho API call
        contents = [
            prompt,
            image,
            f"Câu hỏi: {question}",
            f"Câu trả lời: {answer}"
        ]
        
        # Gọi Gemini API với delay để tránh rate limit
        time.sleep(4)  # Delay 4 giây để tránh limit 15 RPM
        response = client.models.generate_content(
            model='gemini-2.5-flash-lite-preview-06-17', #'gemini-2.5-flash-lite-preview-06-17'
            contents=contents,
            config=types.GenerateContentConfig(
                temperature=0.4
            )
        )
        result = response.text.strip()
        
        # Làm sạch kết quả
        result = re.sub(r'[^\w\s]', '', result).strip()
        
        return result
    
    except Exception as e:
        print(f"Lỗi khi phân loại câu hỏi: {e}")
        return "Unknown"

def process_batch(df: pd.DataFrame, batch_size: int = 10) -> pd.DataFrame:
    """
    Xử lý phân loại theo batch để tránh vượt quá rate limit
    
    Args:
        df: DataFrame chứa dữ liệu
        batch_size: Kích thước batch
    
    Returns:
        DataFrame đã được phân loại
    """
    
    # Tạo các cột mới
    df['Bloom'] = ''
    df['Level'] = ''
    df['Type'] = ''
    
    # Thư mục gốc chứa ảnh
    root_folder = r'C:\Users\Acer\OneDrive\Desktop\NCKH\Dataset\ViVQA4Edu'
    
    total_rows = len(df)
    print(f"Bắt đầu phân loại {total_rows} câu hỏi...")
    
    for i in range(0, total_rows, batch_size):
        batch_end = min(i + batch_size, total_rows)
        batch_df = df.iloc[i:batch_end]
        
        print(f"Đang xử lý batch {i//batch_size + 1}: câu hỏi {i+1}-{batch_end}")
        
        for idx, row in batch_df.iterrows():
            question = row['Question']
            answer = row['Answer']
            
            # Tạo đường dẫn ảnh
            file_name = row['ImageID'] + '.png'
            subfolder_name = file_name.split('_')[0]
            full_path = os.path.join(root_folder, subfolder_name, file_name)
            
            print(f"  Phân loại câu hỏi {idx+1}: {question[:50]}...")
            print(f"    Ảnh: {full_path}")
            
            # Phân loại Bloom Taxonomy
            bloom_result = classify_question(full_path, question, answer, 'bloom')
            df.at[idx, 'Bloom'] = bloom_result
            
            # Phân loại Level
            level_result = classify_question(full_path, question, answer, 'level')
            df.at[idx, 'Level'] = level_result
            
            # Phân loại Type
            type_result = classify_question(full_path, question, answer, 'type')
            df.at[idx, 'Type'] = type_result
            
            print(f"    Bloom: {bloom_result}, Level: {level_result}, Type: {type_result}")
        
        print(f"Hoàn thành batch {i//batch_size + 1}")
        print(f"Đã xử lý {batch_end}/{total_rows} mẫu")
        # Delay ngắn giữa các batch
        time.sleep(1)
    
    return df

print("Đã định nghĩa các hàm phân loại!")

Đã định nghĩa các hàm phân loại!


In [34]:
# Đọc dữ liệu từ file CSV
csv_file_path = r"C:\Users\Acer\OneDrive\Desktop\NCKH\Dataset\ViVQA4Edu-CSV\Verify\Verify_Convert_80.csv"


# Đọc dữ liệu
df = pd.read_csv(csv_file_path)
print(f"Đã đọc {len(df)} dòng dữ liệu từ file {csv_file_path}")
print("\nCấu trúc dữ liệu:")
print(df.head())

# Kiểm tra các cột cần thiết
required_columns = ['ImageID', 'Question', 'Answer']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"Thiếu các cột: {missing_columns}")
    print(f"Các cột hiện tại: {df.columns.tolist()}")
else:
    print("Dữ liệu đã sẵn sàng để phân loại!")

Đã đọc 18838 dòng dữ liệu từ file C:\Users\Acer\OneDrive\Desktop\NCKH\Dataset\ViVQA4Edu-CSV\Verify\Verify_Convert_80.csv

Cấu trúc dữ liệu:
                  ImageID                                           Question  \
0  Education_000000000176        Các bạn học sinh đang làm gì trong lớp học?   
1       Life_000000000565             Trong hình 1, bé gái đang chơi trò gì?   
2       Life_000000000568                           Khăn trải bàn có màu gì?   
3       Life_000000000568                          Trên bàn ăn có mấy người?   
4       Life_000000000784  Người đàn ông đang làm gì với chiếc điều khiển...   

                                              Answer  
0  Các bạn học sinh đang thực hiện các hoạt động ...  
1  Bé gái đang chơi trò nhảy dây. Cô bé cầm dây m...  
2          Khăn trải bàn có màu vàng kẻ ô vuông.\r\n  
3  Trên bàn ăn có 4 người, gồm bố, mẹ, con trai v...  
4  Người đàn ông đang sử dụng điều khiển từ xa để...  
Dữ liệu đã sẵn sàng để phân loại!


In [35]:
# Thực hiện phân loại
print("Bắt đầu quá trình phân loại...")
print("=" * 60)

# Lấy một subset nhỏ để test trước (có thể thay đổi)
# df_sample = df.head(20)  # Uncomment để test với 5 dòng đầu
df_sample = df[0:200]  # Xử lý toàn bộ dữ liệu

# Thực hiện phân loại
result_df = process_batch(df_sample, batch_size=5)

print("=" * 60)
print("Hoàn thành quá trình phân loại!")
print("\nKết quả phân loại:")
print(result_df[['Question', 'Answer', 'Bloom', 'Level', 'Type']].head(10))

Bắt đầu quá trình phân loại...
Bắt đầu phân loại 200 câu hỏi...
Đang xử lý batch 1: câu hỏi 1-5
  Phân loại câu hỏi 1: Các bạn học sinh đang làm gì trong lớp học?...
    Ảnh: C:\Users\Acer\OneDrive\Desktop\NCKH\Dataset\ViVQA4Edu\Education\Education_000000000176.png


C:\Users\Acer\AppData\Local\Temp\ipykernel_17092\2576674825.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bloom'] = ''
C:\Users\Acer\AppData\Local\Temp\ipykernel_17092\2576674825.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Level'] = ''
C:\Users\Acer\AppData\Local\Temp\ipykernel_17092\2576674825.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

Lỗi khi phân loại câu hỏi: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash-lite'}, 'quotaValue': '1000'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '27s'}]}}
Lỗi khi phân loại câu hỏi: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current 

KeyboardInterrupt: 

In [ ]:
# Phân tích kết quả phân loại
print("PHÂN TÍCH KẾT QUẢ PHÂN LOẠI")
print("=" * 60)

# Phân tích Bloom Taxonomy
print("\n1. PHÂN LOẠI BLOOM TAXONOMY:")
bloom_counts = result_df['Bloom'].value_counts()
print(bloom_counts)
print(f"Tổng số nhãn Bloom: {bloom_counts.sum()}")

# Phân tích Level
print("\n2. PHÂN LOẠI LEVEL (LỚP HỌC):")
level_counts = result_df['Level'].value_counts()
print(level_counts)
print(f"Tổng số nhãn Level: {level_counts.sum()}")

# Phân tích Type
print("\n3. PHÂN LOẠI TYPE (DẠNG CÂU HỎI):")
type_counts = result_df['Type'].value_counts()
print(type_counts)
print(f"Tổng số nhãn Type: {type_counts.sum()}")

# Hiển thị một số ví dụ
print("\n4. MỘT SỐ VÍ DỤ PHÂN LOẠI:")
print("-" * 60)
for i, row in result_df.head(3).iterrows():
    print(f"Câu hỏi {i+1}: {row['Question']}")
    print(f"Câu trả lời: {row['Answer']}")
    print(f"Bloom: {row['Bloom']}, Level: {row['Level']}, Type: {row['Type']}")
    print("-" * 60)

PHÂN TÍCH KẾT QUẢ PHÂN LOẠI

1. PHÂN LOẠI BLOOM TAXONOMY:
Bloom
Remember      213
Unknown       191
Understand     95
Analyze         1
Name: count, dtype: int64
Tổng số nhãn Bloom: 500

2. PHÂN LOẠI LEVEL (LỚP HỌC):
Level
1          191
Unknown    191
2          112
3            6
Name: count, dtype: int64
Tổng số nhãn Level: 500

3. PHÂN LOẠI TYPE (DẠNG CÂU HỎI):
Type
Unknown    191
OBJ        146
ACT         59
CNT         41
LOC         31
TXT         20
INF         12
Name: count, dtype: int64
Tổng số nhãn Type: 500

4. MỘT SỐ VÍ DỤ PHÂN LOẠI:
------------------------------------------------------------
Câu hỏi 1: Các bạn học sinh đang làm gì trong lớp học?
Câu trả lời: Các bạn học sinh đang thực hiện các hoạt động khác nhau trong lớp học như lau dọn bàn ghế, quét sàn và cầm sách vở, cho thấy lớp học đang được chuẩn bị cho buổi học hoặc một hoạt động nào đó.

Bloom: Understand, Level: 2, Type: ACT
------------------------------------------------------------
Câu hỏi 2: Trong hình 1

In [ ]:
# Lưu kết quả vào file CSV
output_file = "d:/IT/GITHUB/FinalProject_DataLabeling/Verify_Convert_80_Classified_1.csv"

# Lưu file
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Đã lưu kết quả vào file: {output_file}")

# Hiển thị thông tin file đã lưu
print(f"\nThông tin file đã lưu:")
print(f"- Đường dẫn: {output_file}")
print(f"- Số dòng: {len(result_df)}")
print(f"- Các cột: {result_df.columns.tolist()}")

# Kiểm tra file đã được lưu
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"- Kích thước file: {file_size} bytes")
    print("✅ File đã được lưu thành công!")
else:
    print("❌ Có lỗi khi lưu file!")

# Hiển thị mẫu dữ liệu cuối cùng
print("\nMẫu dữ liệu cuối cùng:")
print(result_df[['ImageID', 'Question', 'Answer', 'Bloom', 'Level', 'Type']].head())

Đã lưu kết quả vào file: d:/IT/GITHUB/FinalProject_DataLabeling/Verify_Convert_80_Classified_1.csv

Thông tin file đã lưu:
- Đường dẫn: d:/IT/GITHUB/FinalProject_DataLabeling/Verify_Convert_80_Classified_1.csv
- Số dòng: 500
- Các cột: ['ImageID', 'Question', 'Answer', 'Bloom', 'Level', 'Type']
- Kích thước file: 101279 bytes
✅ File đã được lưu thành công!

Mẫu dữ liệu cuối cùng:
                  ImageID                                           Question  \
0  Education_000000000176        Các bạn học sinh đang làm gì trong lớp học?   
1       Life_000000000565             Trong hình 1, bé gái đang chơi trò gì?   
2       Life_000000000568                           Khăn trải bàn có màu gì?   
3       Life_000000000568                          Trên bàn ăn có mấy người?   
4       Life_000000000784  Người đàn ông đang làm gì với chiếc điều khiển...   

                                              Answer       Bloom Level Type  
0  Các bạn học sinh đang thực hiện các hoạt động ...  Unde